# $p$-values for Standarised $\beta$'s, partial correlation coefficient, and semi-partial correlation coefficient

__Thesis.__ The $p$-values for the quantities in the title should be identical, since these are simply rescalings of the same effect. See this SE answer:https://stats.stackexchange.com/a/76819/277115.

In what follows, I will refer to the following sources, and work with the `duncan_prestige` dataset.

Kim: https://journals.sagepub.com/doi/10.3102/1076998610396901

Aloe and Thompson: https://www.journals.uchicago.edu/doi/abs/10.5243/jsswr.2013.24

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

from pingouin import partial_corr
from scipy.stats import t

## Perform OLS

In [2]:
# import "duncan_prestige" dataset:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData").data.drop("type", axis=1)

# standardise Y and X
Y = (duncan_prestige['income'] - duncan_prestige['income'].mean()) / duncan_prestige['income'].std()
X = (duncan_prestige[['education', 'prestige']] - duncan_prestige[['education', 'prestige']].mean()) / duncan_prestige[['education', 'prestige']].std()

# perform regression
model_ols = sm.OLS(Y,X)
results_ols = model_ols.fit()

In [3]:
results_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 income   R-squared (uncentered):                   0.702
Model:                            OLS   Adj. R-squared (uncentered):              0.688
Method:                 Least Squares   F-statistic:                              50.73
Date:                Fri, 28 May 2021   Prob (F-statistic):                    4.84e-12
Time:                        19:12:12   Log-Likelihood:                         -36.081
No. Observations:                  45   AIC:                                      76.16
Df Residuals:                      43   BIC:                                      79.78
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
education      0.0393      0.159      0.247      0.806      -0.281       0.360
prestige       0.8043      0.159      5.063      0.000       0.484       1.125
==============================================================================
Omnibus:                        9.200   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.010   Jarque-Bera (JB):               21.265
Skew:                           0.075   Prob(JB):                     2.41e-05
Kurtosis:                       6.364   Cond. No.                         3.54
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Partial Correlation Coefficient

We first simply employ `pingouin` to compute the coefficient and corresponding $p$-values:

In [4]:
# initialise dictionary
partial_corr_dict = {}

# compute correlation coefficient
for predictor in ["education", "prestige"]:
    
    partial_corr_dict[predictor] = partial_corr(data=duncan_prestige,
                                                y="income",
                                                x=predictor,
                                                covar=duncan_prestige.columns.drop([predictor, "income"]).to_list()
                                                )[["r", "p-val"]].reset_index().drop("index", axis=1).values[0]
    
# convert dictionary to dataframe
partial_corr_df = pd.DataFrame.from_dict(partial_corr_dict,
                                         orient="index",
                                         columns=["r", "p_val_pingouin"]
                                        ).sort_values(
    "r", key=abs, ascending=False)

In [5]:
partial_corr_df

,r,p_val_pingouin
prestige,0.611101,0.000011
education,0.037690,0.808084


We compare with the OLS $p$-values:

In [6]:
results_ols.pvalues

education    0.805831
prestige     0.000008
dtype: float64

The numbers are close, but not exact.

So we now compute by hand using the same equation as `pingouin` does. This is equation (2.8) in Kim, which is equivalent to equation (2) in Aloe and Thompson
$$
    t = r \sqrt{\frac{\text{df}}{1 - r^2}}\ .
$$
We only change $\text{df}$ to
$$
    \text{df} = n - g - 1\ ,
$$
where $n$ is the sample size, and $g$ is the number of covariates.

In [7]:
N = Y.shape[0] # sample size
K = X.shape[1] # number of predictors
G = K - 1 # number of covariates

DF = N - G - 1

# compute t statistic and p value
partial_corr_df["t_stat_true"] = partial_corr_df["r"] * np.sqrt(DF / (1 - partial_corr_df["r"]**2))
partial_corr_df["p_val_true"] = 2 * t.sf(partial_corr_df["t_stat_true"].abs(),
                                         df=DF)

In [8]:
partial_corr_df

,r,p_val_pingouin,t_stat_true,p_val_true
prestige,0.611101,0.000011,5.062523,0.000008
education,0.037690,0.808084,0.247328,0.805831


We now have exact agreement with the OLS results.

This means that the $\text{df}$ in both Kim, and hence `ppcor`, (to wit, $n-g-2$), and in Aloe and Thompson ($n-g$), see the text under equation 2) are incorrect (see the numerical confirmation below). A moment's thought shows this to be logical: if we are computing a correlation of two random variables with a sample of size $n$, we set $\text{df} = n - 1 - 1$; for the case of a partial correlation, we simply replace one of the $1$'s in the above formula by $g$ for the co-variates. It appears as though both Kim and Aloe simply made clerical errors, the former overcounting by 1, and the latter undercounting by 1

In [9]:
DF_KIM = DF - 1
partial_corr_df["t_stat_kim"] = partial_corr_df["r"] * np.sqrt(DF_KIM / (1 - partial_corr_df["r"]**2))
partial_corr_df["p_val_kim"] = 2 * t.sf(partial_corr_df["t_stat_kim"].abs(),
                                        df=DF_KIM)

DF_ALOE = DF + 1
partial_corr_df["t_stat_aloe"] = partial_corr_df["r"] * np.sqrt(DF_ALOE / (1 - partial_corr_df["r"]**2))
partial_corr_df["p_val_aloe"] = 2 * t.sf(partial_corr_df["t_stat_aloe"].abs(),
                                         df=DF_ALOE)

In [10]:
partial_corr_df

,r,p_val_pingouin,t_stat_true,p_val_true,t_stat_kim,p_val_kim,t_stat_aloe,p_val_aloe
prestige,0.611101,0.000011,5.062523,0.000008,5.003310,0.000011,5.121051,0.000006
education,0.037690,0.808084,0.247328,0.805831,0.244435,0.808084,0.250187,0.803607


## Semi-partial Correlation Coefficient

We now consider the semi-partial correlation coefficient, which I again compute using `pingouin`.

Given the $\text{df}$ error noted above, I avoid computing the $t$ statistic and $p$-value in `pingouin` and do it from the beginning by hand. This time however, I have two formulae for the $t$-statistic which are not commensurate: equation (2.8) in Kim as above (he uses the same formula for both the partial and semi-partial correlation coefficient):
$$
    t = r \sqrt{\frac{\text{df}}{1 - r^2}}\ ,
$$
versus equation (5) in Aloe and Thompson
$$
    t = r \sqrt{\frac{\text{df}}{1 - R^2}}\ .
$$
where each author uses their respective definitions of $\text{df}$ as noted above. In Aloe and Thompson's formula, $R^2$ denotes the squared explained variance for the _full_ model. Hence, these two formulae will only agree in the case of a single predictor.

We now compute these as above, down to the respective author's $\text{df}$:

In [11]:
# initialise dictionary
semi_partial_corr_dict = {}

# compute correlation coefficient for predictors
for predictor in ["education", "prestige"]:
    
    semi_partial_corr_dict[predictor] = partial_corr(data=duncan_prestige,
                                                     y="income",
                                                     x=predictor,
                                                     x_covar=duncan_prestige.columns.drop([predictor, "income"]).to_list()
                                                     )[["r"]].reset_index().drop("index", axis=1).values[0]
    

# convert dictionary to dataframe
semi_partial_corr_df = pd.DataFrame.from_dict(semi_partial_corr_dict,
                                              orient="index",
                                              columns=["r"]
                                              ).sort_values(
    "r", key=abs, ascending=False)


# compute t stat and p value via kim
semi_partial_corr_df["t_stat_kim"] = semi_partial_corr_df["r"] * np.sqrt(DF_KIM / (1 - semi_partial_corr_df["r"]**2))
semi_partial_corr_df["p_val_kim"] = 2 * t.sf(semi_partial_corr_df["t_stat_kim"].abs(),
                                             df=DF_KIM)

# compute t-stat and p-value via aloe
semi_partial_corr_df["t_stat_aloe"] = semi_partial_corr_df["r"] * np.sqrt(DF_ALOE / (1 - results_ols.rsquared))
semi_partial_corr_df["p_val_aloe"] = 2 * t.sf(semi_partial_corr_df["t_stat_aloe"].abs(),
                                              df=DF_ALOE)

In [12]:
semi_partial_corr_df

,r,t_stat_kim,p_val_kim,t_stat_aloe,p_val_aloe
prestige,0.421208,3.009758,0.004409,5.121051,0.000006
education,0.020578,0.133389,0.894523,0.250187,0.803607


A comparion to OLS results:

In [13]:
results_ols.pvalues

education    0.805831
prestige     0.000008
dtype: float64

We see that Kim's computation quite far away from the true value, while Aloe's is much closer. So we modify Aloe's calculation to use the $\text{df}$ that we argued for above.

In [14]:
semi_partial_corr_df["t_stat_true"] = semi_partial_corr_df["r"] * np.sqrt(DF / (1 - results_ols.rsquared))
semi_partial_corr_df["p_val_true"] = 2 * t.sf(semi_partial_corr_df["t_stat_true"].abs(),
                                              df=DF)

In [15]:
semi_partial_corr_df

,r,t_stat_kim,p_val_kim,t_stat_aloe,p_val_aloe,t_stat_true,p_val_true
prestige,0.421208,3.009758,0.004409,5.121051,0.000006,5.062523,0.000008
education,0.020578,0.133389,0.894523,0.250187,0.803607,0.247328,0.805831


This works.

## Conclusion

1. For both the partial and semi-partial correlation coefficients, the correct $\text{df}$ is $\text{df}=n-g-1$, where $g$ is the number of covariates. This is the only thing that needs to change in the `pingouin` implementation for the partial-correlation coefficient: the $t$-statistic is computed correctly.
2. In the case of the semi-partial correlation coefficient, Kim's formula for the $t$-statistic is incorrect: the correct formulat is equation (5) in Aloe and Thompson (but with $\text{df}$ as above).